In [86]:
import datetime as dt
from lxml import html
from bs4 import BeautifulSoup
import urllib
import sys
import requests
import operator
from datetime import timedelta
import csv
import pandas as pd
import numpy as np
import math
teams = pd.read_csv('teams_table.csv')
year1617 = pd.read_csv('../GamesData/Strength of Schedule/season1617.csv', parse_dates=['Date'])
year1718 = pd.read_csv('../GamesData/Strength of Schedule/season1718.csv', parse_dates=['Date'])
year1819 = pd.read_csv('../GamesData/Strength of Schedule/season1819.csv', parse_dates=['Date'])
year1920 = pd.read_csv('../GamesData/Strength of Schedule/season1920.csv', parse_dates=['Date'])
curr_metrics = pd.read_csv('1-30-teammetrics.csv')

In [87]:
def fill_games_list(games_table,game_list,yesterday):
    for x in games_table:

        if x.find("tbody").findAll("tr")[0].find("a").has_attr('href') and x.find("tbody").findAll("tr")[1].find("a").has_attr('href'):
            
            winner = x.find("tbody").find("tr",{"class":"winner"}).find("a").text
            winner = remove_char(winner)
            winner_score = x.find("tbody").find("tr",{"class":"winner"}).find("td",{"class":"right"}).text
            loser = x.find("tbody").find("tr",{"class":"loser"}).find("a").text
            loser = remove_char(loser)
            loser_score = x.find("tbody").find("tr",{"class":"loser"}).find("td",{"class":"right"}).text
            home_team =x.find("tbody").findAll("tr")[1].find("a").text
            home_team = remove_char(home_team)
            away_team =x.find("tbody").findAll("tr")[0].find("a").text
            away_team = remove_char(away_team)
            
            if winner in teams['sr_games_name'].unique() and loser in teams['sr_games_name'].unique():
                winnerID = teams[teams['sr_games_name'] == winner]['teamID'].values[0]
                loserID = teams[teams['sr_games_name'] == loser]['teamID'].values[0]
                homeID = teams[teams['sr_games_name'] == home_team]['teamID'].values[0]
                awayID = teams[teams['sr_games_name'] == away_team]['teamID'].values[0]
            else:
                print(winner,loser)
            try:
                game_list.append((yesterday,winnerID,winner_score,loserID,loser_score,homeID,awayID))
            except:
                print((yesterday,winner,winner_score,loser,loser_score,home_team,away_team))
        else:
            pass
        
def get_yest_games():
    game_list = []
    for x in range(150,0,-1):
        yesterday = dt.date.today()- timedelta(days=x)
        if yesterday == dt.date(2020, 1, 20):
            continue
        SR_games = "https://www.sports-reference.com/cbb/boxscores/index.cgi?month="+str(yesterday.month)+"&day="+str(yesterday.day)+"&year="+str(yesterday.year)
#         print(SR_games)
#         try:
        games = requests.get(SR_games)
        soup_games = BeautifulSoup(games.content, 'lxml')
        games_table=soup_games.findAll("div",{"class":"game_summary nohover"})
        fill_games_list(games_table,game_list,yesterday)
#         except:
#             print("ERROR!")
    return game_list

def remove_char(teamname):
    if "-" in teamname or "(" in teamname or ")" in teamname or "." in teamname:
        teamname = teamname.replace("-", " ")
        teamname = teamname.replace("(", "")
        teamname = teamname.replace(")", "")
        teamname = teamname.replace(".", "")
    return teamname

In [88]:
# for x in range(150,0,-1):
#         yesterday = dt.date.today()- timedelta(days=x)
#         print(yesterday)

In [67]:
games = get_yest_games()

In [129]:
curr_season = pd.DataFrame(games,columns=['Date','Winner','WinnerScore','Loser','LoserScore','Home','Away'])
curr_season['WinnerScore'] = pd.to_numeric(curr_season['WinnerScore'])
curr_season['LoserScore'] = pd.to_numeric(curr_season['LoserScore'])

In [130]:
frames = [year1617,year1718,year1819,year1920]
allyears = pd.concat(frames).drop(columns=['Unnamed: 0'])
allyears = allyears.drop_duplicates(subset=['Date', 'Home'],keep='last')
# full = allyears[['Home','Away','WinnerHome','KenPomVal_Away','BPIval_Away','SRS_Away','KenPomVal_Home','BPIval_Home','SRS_Home']]


### curr_season is the games that have happened in the 20-21 season
### curr_metrics is the metrics of the teams in the 20-21 season to join curr_season with
### allyears is the 2016-2020 seasons already joined with their metrics

#### 

#### Comparing Win Percentage of home team

In [131]:
curr_season['WinnerHome'] = np.where(curr_season['Winner']==curr_season['Home'],1,0)

In [132]:
print("2020-21 Season")
print("Number of Games won by home team:",curr_season['WinnerHome'].sum())
print("Number of Games played:",len(curr_season))
print("Home Win Percentage:","{:.0%}".format(curr_season['WinnerHome'].sum()/len(curr_season)))

2020-21 Season
Number of Games won by home team: 1362
Number of Games played: 2183
Home Win Percentage: 62%


In [133]:
print("2016-20 Seasons")
print("Number of Games won by home team:",allyears['WinnerHome'].sum())
print("Number of Games played:",len(allyears))
print("Home Win Percentage:","{:.0%}".format(allyears['WinnerHome'].sum()/len(allyears)))

2016-20 Seasons
Number of Games won by home team: 14178
Number of Games played: 21979
Home Win Percentage: 65%


#### Comparing Point Differential

In [134]:
curr_season['HomePtDiff'] = np.where(curr_season['WinnerHome']==1,curr_season['WinnerScore']-curr_season['LoserScore'],curr_season['LoserScore']-curr_season['WinnerScore'])

In [144]:
curr_season['abs_pt_dif'] = curr_season['HomePtDiff'].abs()

In [145]:
curr_season

,Date,Winner,WinnerScore,Loser,LoserScore,Home,Away,WinnerHome,HomePtDiff,abs_pt_dif
0,2020-11-25,335,76,27,67,27,335,0,-9,9
1,2020-11-25,336,89,302,54,336,302,1,35,35
2,2020-11-25,128,97,204,67,128,204,1,30,30
3,2020-11-25,351,77,79,67,351,79,1,10,10
4,2020-11-25,121,122,203,60,121,203,1,62,62
...,...,...,...,...,...,...,...,...,...,...
2178,2021-01-29,353,79,246,70,353,246,1,9,9
2179,2021-01-29,91,64,284,63,91,284,1,1,1
2180,2021-01-29,329,73,288,60,329,288,1,13,13
2181,2021-01-29,15,83,323,75,15,323,1,8,8


In [146]:
curr_season['abs_pt_dif'].mean()

12.832798900595511

In [141]:
allyears['HomePtDiff'] = np.where(allyears['WinnerHome']==1,allyears['Win_Score']-allyears['Lose_Score'],allyears['Lose_Score']-allyears['Win_Score'])

In [147]:
allyears['abs_pt_dif'] = allyears['HomePtDiff'].abs()
allyears

,Date,Win_Score,Lose_Score,Home,Away,WinnerHome,KenPomRank_Away,KenPomVal_Away,BPIrank_Away,BPIval_Away,...,papg_Home,Pace_Home,Three_Par_Home,True_Shoot_Home,Tov_Home,Opp_Three_Par_Home,Opp_True_Shoot_Home,Opp_Tov_Home,HomePtDiff,abs_pt_dif
0,2016-11-11,94,49,Duke,Marist,1,312.0,-13.63,305.0,-8.0,...,70.2,69.9,0.383,0.588,14.2,0.288,0.510,14.9,45,45
1,2016-11-11,87,64,Kentucky,Stephen F. Austin,1,233.0,-6.19,262.0,-5.2,...,71.5,74.3,0.316,0.564,13.4,0.333,0.507,16.9,23,23
2,2016-11-11,103,99,Kansas,Indiana,0,44.0,14.49,32.0,10.6,...,71.9,71.4,0.359,0.583,15.1,0.342,0.515,16.1,-4,4
3,2016-11-11,88,48,Villanova,Lafayette,1,330.0,-16.79,330.0,-10.9,...,62.7,65.6,0.433,0.612,15.0,0.400,0.506,17.6,40,40
4,2016-11-11,91,77,Oregon,Army,1,230.0,-6.04,253.0,-4.7,...,65.8,68.8,0.388,0.582,14.7,0.378,0.494,16.6,14,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5298,2020-03-11,74,69,Alabama-Birmingham,Texas-San Antonio,1,210.0,-3.84,230.0,-3.5,...,67.3,67.2,0.300,0.537,19.0,0.409,0.514,16.2,5,5
5299,2020-03-11,71,69,Oregon State,Utah,1,114.0,5.56,117.0,3.3,...,71.2,66.6,0.349,0.559,15.3,0.364,0.529,16.2,2,2
5300,2020-03-11,86,73,Arkansas,Vanderbilt,1,169.0,0.14,165.0,0.3,...,75.8,71.4,0.399,0.553,14.4,0.333,0.516,19.2,13,13
5301,2020-03-11,77,70,Arizona,Washington,1,54.0,12.67,70.0,7.0,...,76.4,70.3,0.347,0.546,14.5,0.377,0.498,18.1,7,7


In [148]:
allyears['abs_pt_dif'].mean()

12.038400291187042